In [ ]:
#@title Shift+Enter키 혹은 왼쪽의 Play 버튼을 클릭해서 AI 모델을 활성화 합니다. 
#@markdown * 구글 계정이 필요합니다. 
#@markdown * 로그인 후 "Run Anyway"버튼 클릭합니다. 
print('1. Model Downloading')
from os.path import join

modelPath='/content'
dataPath='/content'

# from google.colab import drive
# isMount=!df |grep /content/drive
# if len(isMount) < 1 : drive.mount('/content/drive', force_remount=False)


!gdown -q --id "1UvDjOO7Gmh-F6h3YuYZHQCBuUH_ujxZO" >> /tmp/null
dPath="/content/term_job_ctg.tsv"

!gdown -q --id "1gurNvZ8fg9dFww7rlC4gfP-Kd_OdNZh7" >> /tmp/null
mPath="/content/jobCorrect.csv"

#bertmodel
!gdown -q --id "1hHd8COvWlGCeJE31L4jvrhIYqjop6B0L" >> /tmp/null
#vocab
!gdown -q --id "1-6Ej2ez6COAlzUj_izZe1Ybmnrr2gTMq" >> /tmp/null

#job_ctg_decoder
!gdown -q --id "1-9YTmqsbd0MZWAsIwXBgWTJsIMiuxOf-" >> /tmp/null

#job_ctg_to_profession
!gdown -q --id "1-UI8RKmTbXWk4tM1nDTJBto1k78feuMF" >> /tmp/null

#bert_job_ctg_2nd
!gdown -q --id "16_Z8b2FHw7orLJOClx6VJ-kRE0jZa-X2" >> /tmp/null


!pip install mxnet >> /tmp/null
!pip install gluonnlp pandas tqdm  >> /tmp/null
!pip install sentencepiece  >> /tmp/null
!pip install transformers==3.0.2  >> /tmp/null
!pip install torch  >> /tmp/null
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master  >> /tmp/null

print('2. Model Loading')
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
import pickle
from os.path import join

#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model


#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
#GPU 사용
device = torch.device("cuda:0")



from os.path import join, isdir, isfile
import os.path


from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets


import warnings
warnings.filterwarnings(action='ignore')

isModelActivated=False

#  함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
def get_1d(l):
  ll=[]
  for i in l:
    for j in i:
      ll.append(j)
  return ll

# get_1d([[1],[2]])  
def calc_accuracy2(x,y):
  xx=get_1d(x)
  yy=get_1d(y)
  xl=len(xx)
  cnt=0
  if xl==len(yy):
    for i in range(xl):
      if xx[i] == yy[i]:
        cnt+=1
    return cnt/xl

# calc_accuracy2([[1,2],[3,4]],[[1],[2,3,1]])

def isEqual(X,Y):
  max_vals, max_indices = torch.max(X, 1)
  if max_indices.cpu().numpy()[0] == Y:
    return 1
  else:
    return 0

class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

def corderReversor(corder):
  k=corder.keys()
  return { corder[i]:i for i in k}

print('3.Data Loading')
import pandas as pd
df = pd.read_csv(dPath,sep="\t")



data_list = []
for q, label in zip(df['document'], df['label'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

print('4. 추가 데이터 로딩')
from sklearn.model_selection import train_test_split
                                                         
dataset_train=data_list
dataset_test=data_list.copy()

num_classes =df.label.unique().shape[0]


dfm = pd.read_csv(mPath)
dfm_train=dfm[dfm.job_ctg_corr!=0]

import pickle
with open(join(dataPath,'job_ctg_decoder'),'rb') as f:
  job_ctg_decoder=pickle.load(f)



job_ctg_encoder=corderReversor(job_ctg_decoder)
# job_ctg_encoder[-1]=-1

dfm_train=dfm_train.fillna(-1)
# [(job_ctg_encoder[i],i) for i in dfm_train.job_ctg_corr.to_list() if i != -1]
dataset_train2 = [ [r[1].STDNT_HOP_MATR, job_ctg_encoder[ r[1].job_ctg_corr ]] for r in dfm_train.iterrows() if r[1].job_ctg_corr !=-1 ]

dataset_train=dataset_train+dataset_train2
#  Model Loading
with open(join(modelPath,'bertmodel'), "rb") as file_to_read:
  bertmodel = pickle.load(file_to_read)


with  open(join(modelPath,'vocab'), "rb") as file_to_read:
  vocab = pickle.load(file_to_read)

# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 30
max_grad_norm = 1
log_interval = 200
learning_rate =  3e-5

# 토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)


train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)
# BERT 모델 불러오기
num_classes=38
learning_rate =  3e-5

model = BERTClassifier(bertmodel,num_classes=num_classes,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)


def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
# train_dataloader
# Model Load
model = BERTClassifier(bertmodel,  dr_rate=0.5, num_classes=num_classes ).to(device)
model.load_state_dict(torch.load(join(modelPath,"bert_job_ctg_2nd") )) 


print('5. Model Activating')

# Model Implementation 

#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)
# job_ctg_decoder and job_ctg_to_profession  : load
with open(join(dataPath,'job_ctg_decoder'),'rb') as f:
  job_ctg_decoder=pickle.load(f)

with open(join(dataPath,'job_ctg_to_profession'),'rb') as f:
  job_ctg_to_profession=pickle.load(f)



def predict(predict_sentences):
  # predict_sentence="의사"
  dataset_another = [[predict_sentence, '0'] for predict_sentence in predict_sentences]
  # dataset_another = [data]

  another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
  test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)

  model.eval()

  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)

    valid_length= valid_length
    label = label.long().to(device)

    out = model(token_ids, valid_length, segment_ids)


    test_eval=[]
    for i in out:
      logits=i
      logits = logits.detach().cpu().numpy()
      test_eval.append(np.argmax(logits))
  return test_eval

def predictIt(직업설명):
  if isModelActivated:
    if len(직업설명)==0:
      return ""
    else:
      r=[job_ctg_to_profession[job_ctg_decoder[i]] for i in predict([직업설명]) ]
      return r[0]
  else:
    return "위의 셀에서 Shift+Enter키 혹은 왼쪽의 Play 버튼을 클릭해서 AI 모델을 활성화해 주세요."
print('-------------------------------------')
print('AI 모델의 활성화가 완료 되었습니다.')
isModelActivated=True
print('-------------------------------------')

1. Model Downloading
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-g1ha7z7z
2. Model Loading
3.Data Loading
4. 추가 데이터 로딩
using cached model
5. Model Activating
using cached model
-------------------------------------
AI 모델을 활성화가 완료 되었습니다.
-------------------------------------


## 직업정보에서 직업 분류를 예측 


In [ ]:
#@title 직업 연동의 예
job_to_predict=[
  '노래를 부른다.',
  '교회일을 하고 싶다',
  '먹을 것을 만드는 것이 좋다'
]
job_predicted=[job_ctg_to_profession[job_ctg_decoder[i]] for i in predict(job_to_predict) ]


pd.DataFrame({"생활 기록부 상의 희망 직업":job_to_predict, "연동된 직업 정보": job_predicted } )

,생활 기록부 상의 희망 직업,연동된 직업 정보
0,노래를 부른다.,예술·디자인·방송직
1,교회일을 하고 싶다,사회복지·종교직
2,먹을 것을 만드는 것이 좋다,음식 서비스직


In [ ]:

#@title  검색에서 직업 분류명을 예측 
#@markdown * Shift+Enter키 혹은 왼쪽의 Play 버튼을 클릭해서 현재 셀을 활성화해 주세요. 
#@markdown * 직업 설명을 선택해 보세요.
interact(predictIt, 직업설명=[
  '컴퓨터 백신 관련직',
  '음식을 만든다 조리',
  '간호사, 헤어디자이너.',
  '치안을 책임진다.'
]);

interactive(children=(Dropdown(description='직업설명', options=('컴퓨터 백신 관련직', '음식을 만든다 조리', '간호사, 헤어디자이너.', '치안을 책…

In [ ]:
#@markdown ## 직업에 대한 설명을 입력해 보세요. 
#@markdown * Shift+Enter키 혹은 왼쪽의 Play 버튼을 클릭해서 현재 셀을 활성화해 주세요. 
#@markdown * ex) 컴퓨터 백신 관련직
#@markdown * ex) 쉬운일?
interact(predictIt, 직업설명="");

interactive(children=(Text(value='', description='직업설명'), Output()), _dom_classes=('widget-interact',))